## Вакансии на сайте HH

In [65]:
!git clone https://github.com/dnp34/hh_vacancies.git

Cloning into 'hh_vacancies'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 38 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), 89.67 KiB | 3.59 MiB/s, done.


In [66]:
!cp ./hh_vacancies/* ./

In [12]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import re

def get_skills(description, key_skills):
    pp = BeautifulSoup(description, features="html.parser").get_text()
    pp_re = re.findall(r'\s[A-Za-z-?]+', pp)
    its = set(x.strip(' -').lower() for x in pp_re)
    
    skills = set()
    for sk in key_skills:
        skills.add(sk['name'].lower())
        
    for it in its:
        if not any(it in x for x in skills):
            skills.add(it)
            
    return list(skills)

In [55]:
def get_area_id(area_id):
    # загрузка файла с кодами городов
    if exists('area.pkl'):
        with open('area.pkl', mode='rb') as f:
            area = load(f)
    else:
        area = {}
    
    if area_id in area.values():
        return area_id
    
    url = f'https://api.hh.ru/areas/{area_id}'
    res = get(url=url).json()
    area[res['name']] = res['id']
    
    # сохранение файла с результами работы
    with open('area.pkl', mode='wb') as f:
        dump(area, f)
    
    return res['id']

In [63]:
from pprint import pprint
from pickle import dump, load
from os.path import exists
from json import dump as json_dump
from requests import get
from collections import Counter
from bs4 import BeautifulSoup

#from area import get_area_id
#from skills import get_skills

vacancy = 'linux'  # интересующая вакансия
my_city = 'Москва' # город для поиска (all - для всех)
url = 'https://api.hh.ru/vacancies'

# загрузка текущих курсов валют ЦБ РФ
data = get('https://www.cbr-xml-daily.ru/daily_json.js').json()
rate = data['Valute']

# загрузка файла с кодами городов
if exists('area.pkl'):
    with open('area.pkl', mode='rb') as f:
        area = load(f)
else:
    area = {}

print('\n ===== КОДЫ ГОРОДОВ =====')
pprint(area)

p = {'text': vacancy}
r = get(url=url, params=p).json()
count_pages = r['pages']
all_count = len(r['items'])

result = {
        'keywords': vacancy,
        'count': all_count,
        'city': my_city
         }

sal = {'from': [], 'to': []}
skills_all = []

# определяем, сколько будет получено страниц, и проходим по каждой из страниц
print(f'\n🌐 Перебор 5 страниц из {count_pages} на ресурсе: {url}')
for page in range(count_pages):
    if page > 4:
        break
    else:
        print(f'👀 страница: {page+1}')
    p = {'text': vacancy,
         'page': page}
    ress = get(url=url, params=p).json()
    all_count = len(ress['items'])
    result['count'] += all_count
    
    city_count = 0
    for res in ress['items']:
        city_vac = res['area']['name']
        # добавление города из ответа на запроса, если его нет в файле
        if city_vac not in area:
            area[city_vac] = get_area_id(res['area']['id'])
        
        if my_city == 'all' or city_vac == my_city:
            city_count += 1
            res_full = get(res['url']).json()
            skills = get_skills(res_full['description'], res_full['key_skills'])
            skills_all += skills
        
            # обработка зарплаты
            if res_full['salary']:
                code = res_full['salary']['currency']
                #print(f'[{code}]')
                k = 1 if code == 'RUR' else float(rate[code]['Value'])
                sal['from'].append(k * res_full['salary']['from'] if res['salary']['from'] else k * res_full['salary']['to'])
                sal['to'].append(k * res_full['salary']['to'] if res['salary']['to'] else k*res_full['salary']['from'])

# формирование результата
result.update({
    'city_count': city_count,
    'average_from': round(sum(sal['from']) / len(sal['from']), 2),
    'average_to': round(sum(sal['to']) / len(sal['to']), 2),
    'requirements': [{'name': name, 'count': count, 'percent': round((count / result['count'])*100, 2)} 
                      for name, count in Counter(skills_all).most_common(5)]
})

print(f'\n✅ Результаты отбора данных по вакансии [{vacancy}] в городе {my_city}:\n')
pprint(result, sort_dicts=False)

# сохранение файла с результами работы
with open('result.json', mode='w') as f:
    json_dump([result], f)
with open('area.pkl', mode='wb') as f:
    dump(area, f)


 ===== КОДЫ ГОРОДОВ =====
{'Астана': '159',
 'Владивосток': '22',
 'Волгоград': '24',
 'Воронеж': '26',
 'Екатеринбург': '3',
 'Казань': '88',
 'Кипр': '236',
 'Красноярск': '54',
 'Минск': '1002',
 'Москва': '1',
 'Новосибирск': '4',
 'Санкт-Петербург': '2',
 'Сербия': '146',
 'Томск': '90'}

🌐 Перебор 5 страниц из 100 на ресурсе: https://api.hh.ru/vacancies
👀 страница: 1
👀 страница: 2
👀 страница: 3
👀 страница: 4
👀 страница: 5

✅ Результаты отбора данных по вакансии [linux] в городе Москва:

{'keywords': 'linux',
 'count': 120,
 'city': 'Москва',
 'city_count': 14,
 'average_from': 308693.06,
 'average_to': 380305.57,
 'requirements': [{'name': 'linux', 'count': 48, 'percent': 40.0},
                  {'name': 'python', 'count': 27, 'percent': 22.5},
                  {'name': 'nginx', 'count': 23, 'percent': 19.17},
                  {'name': 'bash', 'count': 23, 'percent': 19.17},
                  {'name': 'docker', 'count': 20, 'percent': 16.67}]}


In [64]:
!ls -al *.pkl *.json

-rw-r--r-- 1 root root 475 Mar 30 19:22 area.pkl
-rw-r--r-- 1 root root 422 Mar 30 19:22 result.json
